In [1]:
import warnings                        
warnings.filterwarnings('ignore')  
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
INFO:tensorflow:Disabling eager execution
INFO:tensorflow:Disabling v2 tensorshape
Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Disabling resource variables
INFO:tensorflow:Disabling tensor equality
INFO:tensorflow:Disabling control flow v2


### 학습할 데이터를 만든다.

In [ ]:
xData = [1,2,3,4,5,6,7] # 하루 근무 시간
yData = [25000,55000, 75000, 110000, 128000,180000] # 근무 시간에 따른 매출 금액, 실제값

####  난수를 발생시켜 학습할 데이터의 기울기(가중치)와 y절편(바이어스)을 정한다.
#### 난수를 발생시켜서 작업하는 이유는 과적합을 방지하기 위해서이다.
#### 과적합이란 학습시킨 데이터에서는 높은 정확도를 보이지만 학습시킨 데이터 이외의 데이터에서는 데이터에서는 낮은 정확도를 보이는 문제점을 말한다.